# XNATpy

**XNATpy** is an XNAT client that exposes XNAT objects/functions as python objects/functions. The notebook provides a quick overview of using XNATpy. More information is availble in the [official documentation](https://xnat.readthedocs.io/en/latest/). 

## Setup

If XNATpy is not already installed

In [ ]:
pip install xnat

In [ ]:
import os
import xnat

## Connecting to an XNAT server

When connecting to an XNAT server you start by creating a connection. To avoid storing our credentials in our notebook we can use the XNAT environmental varibales.

In [ ]:
connection = xnat.connect(os.environ['XNAT_HOST'], 
                          user=os.environ['XNAT_USER'], 
                          password=os.environ['XNAT_PASS'])

As of version 0.4.3, XNATpy supports the XNAT/Jupyter envrionment variables. So we can shorten this to

In [ ]:
connection = xnat.connect()

## Exploring your XNAT server

When a session is established, it is fairly easy to explore the data on the XNAT server. The data structure of XNAT is mimicked as Python objects. The connection gives access to a listing of all projects, subjects, and experiments on the server.

In [ ]:
connection.projects

The XNATListing is a special type of mapping in which you can access elements by a primary key (usually the ID or Accession #) and a secondary key (e.g. the label for a subject or experiment). Selection can be performed the same as a Python dict.

In [ ]:
project = connection.projects[0] # or connection.projects["C4KC-KiTS"]
project.subjects

## Building an image viewer

Based on pydicom example [Load CT slices and plot axial, sagittal and coronal images](https://pydicom.github.io/pydicom/stable/auto_examples/image_processing/reslice.html#sphx-glr-auto-examples-image-processing-reslice-py)

This example illustrates loading a scan with XNATpy and pydicom, building a 3D image, and reslicing it in different planes. We will also demostrate the new XNATpy support for the XNAT/Jupyter file system!

In [ ]:
import pydicom
import numpy as np
import matplotlib.pyplot as plt
import sys
import glob
import xnat
import time
import os

from ipywidgets import interact
from scipy import ndimage

Modify this for your project / subject / experiment / dicom scan

In [ ]:
session = xnat.connect(loglevel='INFO')
project = session.projects['C4KC-KiTS']
subject = project.subjects['KiTS-00004']
experiment = subject.experiments['KiTS-00004_CT_1']
scan = experiment.scans['7']

In [ ]:
# load the DICOM files
files = []
for file in scan.files.values():
    if file.uri.endswith("dcm"):
        files.append(pydicom.dcmread(file.open()))
        
print("file count: {}".format(len(files)))

# skip files with no SliceLocation (eg scout views)
slices = []
skipcount = 0
for f in files:
    if hasattr(f, 'SliceLocation'):
        slices.append(f)
    else:
        skipcount = skipcount + 1

print("skipped, no SliceLocation: {}".format(skipcount))

# ensure they are in the correct order
slices = sorted(slices, key=lambda s: s.SliceLocation)

# pixel aspects, assuming all slices are the same
ps = slices[0].PixelSpacing
ss = slices[0].SliceThickness
ax_aspect = ps[1]/ps[0]
sag_aspect = ps[1]/ss
cor_aspect = ss/ps[0]

# create 3D array
img_shape = list(slices[0].pixel_array.shape)
img_shape.append(len(slices))
img3d = np.zeros(img_shape)

# fill 3D array with the images from the files
for i, s in enumerate(slices):
    img2d = s.pixel_array
    img3d[:, :, i] = img2d


def browse_images_ax(img3d, img_shape, ax_aspect):
    n = img_shape[2]
    def view_image(i):
        a = plt.subplot()
        plt.imshow(img3d[:, :, i], cmap=plt.cm.bone, interpolation='nearest')
        a.set_aspect(ax_aspect)
        plt.title('Image: %s' % i)
        plt.show()
    interact(view_image, i=(0,n-1))
    
def browse_images_sag(img3d, img_shape, sag_aspect):
    n = img_shape[1]
    def view_image(i):
        a = plt.subplot()
        rotated_img = ndimage.rotate(img3d[:, i, :], 90)
        plt.imshow(rotated_img, cmap=plt.cm.bone, interpolation='nearest')
        a.set_aspect(1/sag_aspect)
        plt.title('Image: %s' % i)
        plt.show()
    interact(view_image, i=(0,n-1))  
    
def browse_images_cor(img3d, img_shape, cor_aspect):
    n = img_shape[0]
    def view_image(i):
        a = plt.subplot()
        rotated_img = ndimage.rotate(img3d[i, :, :].T, 180)
        plt.imshow(rotated_img, cmap=plt.cm.bone, interpolation='nearest')
        a.set_aspect(cor_aspect)
        plt.title('Image: %s' % i)
        plt.show()
    interact(view_image, i=(0,n-1))  

Notice the log message! XNATpy version 0.4.3 has been updated to support the XNAT/Jupyter integration. XNATpy understands the layout of the XNAT/Jupyter filesystem. If XNATpy can find your data on the filesystem it will use that instead of attempting to download from XNAT.

In [ ]:
browse_images_ax(img3d, img_shape, ax_aspect)
browse_images_sag(img3d, img_shape, sag_aspect)
browse_images_cor(img3d, img_shape, cor_aspect)